In [ ]:
# Imports
import tensorflow as tf
import numpy as np

# Assume raw_notes DataFrame exists from preprocessing
key_order = ['pitch', 'start', 'end', 'velocity']
train_notes = raw_notes[key_order].copy().to_numpy()

# Convert to tensor dataset
notes_ds = tf.data.Dataset.from_tensor_slices(train_notes)

# Create sequences
def create_seq(dataset, seq_length=25):
    seq_ds = dataset.window(seq_length + 1, shift=1, drop_remainder=True)
    seq_ds = seq_ds.flat_map(lambda window: window.batch(seq_length + 1))
    seq_ds = seq_ds.map(lambda window: (window[:-1], window[-1]))
    return seq_ds

seq_length = 25
seq_ds = create_seq(notes_ds, seq_length)
seq_ds = seq_ds.shuffle(buffer_size=10000).batch(128).prefetch(tf.data.AUTOTUNE)

# Model architecture
def build_model(seq_length, num_features, lstm_units=128):
    inputs = tf.keras.Input(shape=(seq_length, num_features))
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences=True))(inputs)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units))(x)

    pitch = tf.keras.layers.Dense(128, activation='softmax', name='pitch')(x)
    step = tf.keras.layers.Dense(1, name='step')(x)
    duration = tf.keras.layers.Dense(1, name='duration')(x)
    velocity = tf.keras.layers.Dense(1, name='velocity')(x)

    return tf.keras.Model(inputs, outputs=[pitch, step, duration, velocity])

model = build_model(seq_length=25, num_features=4)

# Compile model
model.compile(
    optimizer='adam',
    loss={
        'pitch': 'sparse_categorical_crossentropy',
        'step': 'mae',
        'duration': 'mae',
        'velocity': 'mae',
    }
)

# Train model
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('models/trained_model.h5', save_best_only=True)
]

history = model.fit(seq_ds, epochs=50, callbacks=callbacks)

# Save model architecture
with open("models/model_architecture.json", "w") as f:
    f.write(model.to_json())
